# How to use LLMs for text analysis

#### 2023-07. By Petter Törnberg. ILLC, University of Amsterdam. p.tornberg@uva.nl

This notebook is associated to a how-to guide that gives a simple introduction to using Large Language Models for text analysis in the social sciences. The guide is aimed at students and researchers who are interested in using LLMs for their quantitative or qualitative social scientific text analysis, but who have limited programming experience. You can find the guide at: https://github.com/cssmodels/howtousellms

The notebook offers the code for the example of analyzing the level of populism in a given political text, but can easily be adapted for your particular text analysis project.

## 1. Signing up for API access

The first step is to sign up to API access with OpenAI. This can be done on platform.openai.com. (See the PDF how-to guide for further instructions.)

You will receive an API key to be used below.

## 2. Installing and loading the relevant libraries

We first need to install and import the relevant libraries: the pandas package for general data processing, and the openai package for interacting with the OpenAI API.

In [ ]:
#Install the libraries
!pip install pandas
!pip install openai
!pip install numpy

In [366]:
#Call the libraries
import pandas as pd
import openai
import numpy as np

#Set the API key. See the how-to guide for furhter instructions
openai.api_key = "YOUR_KEY_HERE"

In [326]:
#We define which model to use throughout
MODEL = 'gpt-4'
MAX_TOKENS = 8000
WAIT_TIME = 0.8 # Wait time between each request. This depends on the rate limit of the model used: GPT-4 needs longer wait time than GPT-3.5.

We can now call the OpenAI API. For instance, we can ask ChatGPT-4 a question:

In [365]:
#Test the API
response = openai.ChatCompletion.create(
  model = 'gpt-4', #Which model to use
  temperature=0.2, #How random is the answer
  max_tokens=120, #How long can the reply be
  messages=[
    {"role": "user", 
    "content": "What is the meaning of life?"}]
)
result = ''
for choice in response.choices:
    result += choice.message.content
print(f"Model answer: '{result}'")


Model answer: 'As an AI, I don't have personal experiences or emotions, but I can tell you that the meaning of life varies from person to person based on their beliefs and values. Some people may believe it's to learn and grow, others may see it as a quest for happiness or fulfillment, and some may find meaning in contributing to the well-being of others. Philosophically, it's a subject of much debate. Some philosophers argue that life has no inherent meaning, and it's up to each individual to create their own purpose.'


If the code above generates an error, you might need to check whether your API key is correct, and whether you have access to the specified model.

## 3. Loading and preparing your data

The next step is to load and prepare the data that we want to analyze. We will load the data into a Pandas dataframe to allow easy processing.

The details of how to open your particular data depends on the structure and format of the data. Pandas offers ways of opening a range of file formats, including CSV and Excel files. You may wish to refer to the Pandas documentation for more details.

In our example, we will use the data from the Global Populism Dataset (https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/LFTQEZ). This data offers a number of texts from politicians, and can be used for validating our method. The texts are provided as .txt files in a folder. We will load all these files into a single dataframe.

In [ ]:
#Loading data from textfiles
import glob
import pandas as pd
import os 

# Define the folder path where the text files are located
folder_path = './global-populism-dataset/speeches_20220427/'

# Use glob to get a list of all *.txt files in the folder
txt_files = glob.glob(folder_path + '/*.txt')

# Create an empty list to store the data
data = []

# Loop through each text file
for file_path in txt_files:
    with open(file_path, 'r',encoding='utf-8',errors='ignore') as file:
        # Read the text from the file
        text = file.read()

        # Get the filename without the directory path
        filename = os.path.basename(file_path)

        # Append the text and filename to the data list
        data.append({'filename': filename, 'text': text})

# Create a dataframe with the data
df = pd.DataFrame(data)

### Filter the data 

You will likely need to filter out and select the data you wish to include. 

In our case, we will filter out texts with non-latin alphabets. While ChatGPT can handle languages with non-latin characters, it is currently more expensive, and there are issues with managing text length. For simplicity, we therefore remove the texts with non-latin alphabet.


In [ ]:

def is_latin_alphabet(text):
    latin_characters = 0
    total_characters = 0

    for char in text:
        if ord(char) >= 0x0000 and ord(char) <= 0x007F:
            latin_characters += 1
        total_characters += 1

    # Check if the majority of characters are Latin alphabet characters
    if latin_characters / total_characters >= 0.9:
        return True
    else:
        return False

df = df[df['text'].apply(is_latin_alphabet)]


### Chunking the texts 

Unlike other NLP methods, not much preprocessing is needed. However, LLMs are only able to process texts that are smaller than their "context window". If our texts are longer than the context window of our model, we have to either split the texts into several smaller chunks and analyze them part by part, or simply truncate the text (not recommended).

The details depend on the model you use and the amount for data. For our example, with the GPT-4-32k model, our speeches all fit in the model window, and we do not need to split the texts. 

However, for pedagogical reasons, we will use the standard 8K GPT-4 model and chunk the text into smaller pieces. If your text is short, such as a tweet, this function will do nothing.

In [231]:
# Example of how to chunk the text into pieces, separated on sentence level.
# To do so, we use the nltk library

In [300]:
!pip install tiktoken
!pip install nltk
!pip install langdetect

In [ ]:
import tiktoken
import nltk
import nltk.data
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')

In [328]:
#This code chunks the text into processable pieces of similar size.
#If the text is longer than allowed in terms of model tokens, we want to split the text into equally sized parts, without splitting any text mid-sentence.
def split_text_into_chunks(text, max_tokens):
    
    #Code the text in gpt coding and calculate the number of tokens
    encoding = tiktoken.encoding_for_model(MODEL)
    nrtokens = len(encoding.encode(text))
        
    if nrtokens < max_tokens:
        return [text]
    
    #how many chunks to split it into?
    num_chunks = np.ceil(nrtokens / max_tokens)

    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    # Calculate the number of words per chunk
    words_per_chunk = len(text.split()) // num_chunks

    # Initialize variables
    chunks = []
    current_chunk = []

    word_counter = 0
    # Iterate through each sentence
    for sentence in sentences:
        # Add the sentence to the current chunk
        current_chunk.append(sentence)
        word_counter += len(sentence.split())

        # Check if the current chunk has reached the desired number of words
        if word_counter >= words_per_chunk:
            # Add the current chunk to the list of chunks
            chunks.append(" ".join(current_chunk))
            word_counter = 0
            # Reset the current chunk
            current_chunk = []

    # Add the remaining sentences as the last chunk
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks



In [329]:
# Maximum number of words per chunk, this depends on the model context window. 
# We set it to a bit lower than the max tokens, to leave space for our instruction and the response.
max_tokens = MAX_TOKENS - 2000
df['text_chunks'] = df['text'].apply(lambda x: split_text_into_chunks(x, max_tokens))

# 4. Prompt engineering

The next step is to formulate a first instructions for analyzing the text. The prompts will be a result of an iterative process through which you develop a formulation of the concept that you wish to capture. 

We here start by drawing on the instructions for human coders from a previous study.

See the how-to guide for details on this process. 

In [330]:
instruction = """Your task is to evaluate the level of populism in a political text. Populism is defined as "an ideology that considers society to be ultimately separated into two homogeneous and antagonistic groups, 'the pure people' versus 'the corrupt elite', and which argues that politics should be an expression of the volonté générale (general will) of the people."
A populist text is characterized by BOTH of the following elements:
- People-centrism: how much does the text focus on "the people" or "ordinary people" as an indivisible or homogeneous community? Does the text promote a politics as the popular will of "the people"?
Appeals to specific subgroups of the population (such as ethnicities, regional groups, classes) are inherently antithetical to populism.
- Anti-elitism: how much does the text focus on "the elite", and to what extent are elites in general described in negative terms? In populist texts, the elite is often described as corrupt, and the juxtaposition between the ordinary people and the elite is cast as a moral struggle between good and bad. 
Criticism of specific elements within an elite is not populist: a populist appeal must regard the elite in its entirety as anathema. 

You should give the text a numeric grade between 0 and 2.
2. The text is very populist and comes very close to the ideal populist discourse.
1. A speech in this category includes strong expressions of all of the populist elements,  but either does not use them consistently or tempers them by including non-populist elements. The text may have a romanticized notion of the people and the idea of a unified popular will, but it avoids bellicose language or any particular enemy.
0. A speech in this category uses few if any populist elements. 
[Answer with a number in the 0-2 range, followed by a semi-colon, and then a brief motivation. For instance: "1.23; The text shows many elements of a populist text." Do not use quotation marks.]
"""

# 5. Calling the LLM and analyzing the results

### 5.1 Call the LLM

We will now write simple functions for calling the API and carry out our analysis request. We will also need to handle possible errors returned from the API.

In [331]:
import time

def analyze_message(text, instruction, model = 'gpt-4', temperature=0.2):
    print(f"Analyzing message...")
    
    response = None
    tries = 0
    failed = True
    
    while(failed):
        try:
            response = openai.ChatCompletion.create(
                model = model, 
                temperature=temperature,
                messages=[
                        {"role": "system", "content": f"'{instruction}'"}, #The system instruction tells the bot how it is supposed to behave
                        {"role": "user", "content": f"'{text}'"} #This provides the text to be analyzed.
                    ]
            )
            failed = False

        #Handle errors.
        #If the API gets an error, perhaps because it is overwhelmed, we wait 10 seconds and then we try again. 
        # We do this 10 times, and then we give up.
        except openai.error.APIError as e:
            print(f"OpenAI API returned an API Error: {e}")
            
            if tries < 10:
                print(f"Caught an APIError: {e}. Waiting 10 seconds and then trying again...")
                failed = True
                tries += 1
                time.sleep(10)
            else:
                print(f"Caught an APIError: {e}. Too many exceptions. Giving up.")
                raise e
                
        except openai.error.ServiceUnavailableError as e:
            print(f"OpenAI API returned an ServiceUnavailable Error: {e}")
            
            if tries < 10:
                print(f"Caught a ServiceUnavailable error: {e}. Waiting 10 seconds and then trying again...")
                failed = True
                tries += 1
                time.sleep(10)
            else:
                print(f"Caught a ServiceUnavailable error: {e}. Too many exceptions. Giving up.")
                raise e
            
        except openai.error.APIConnectionError as e:
            print(f"Failed to connect to OpenAI API: {e}")
            pass
        except openai.error.RateLimitError as e:
            print(f"OpenAI API request exceeded rate limit: {e}")
            pass
        
        #If the text is too long, we truncate it and try again. Note that if you get this error, you probably want to chunk your texts.
        except openai.error.InvalidRequestError as e:
            #Shorten request text
            print(f"Received a InvalidRequestError. Request likely too long; cutting 10% of the text and trying again. {e}")
            time.sleep(5)
            words = text.split()
            num_words_to_remove = round(len(words) * 0.1)
            remaining_words = words[:-num_words_to_remove]
            text = ' '.join(remaining_words)
            failed = True
        
        except Exception as e:
            print(f"Caught unhandled error.")
            pass
            
    result = ''
    for choice in response.choices:
        result += choice.message.content
    
    return result 


    

### 5.2 Parse response 

The LLM will return a text message. We need to parse this response so that we can use it for further analysis. The details of this function will depend on how you asked the API to respond in your instruction (see above). In our case, we asked the LLM to return a list of numbers, followed by a motivation.

In [332]:
def parse_result(result):
    #The LLMs at times surround their answers with quotation marks, even if you explicitly tell them not to. If so, we remove them her.
    result = result.strip("'\"") 
    try:
        #We asked the LLM to start with a number, followed by a semi-colon, followed by the motivation. We assume this format in the response here.
        return result.split(';', 2) #Split by ';' and use first part as numeric answer, second part as motivation
    except Exception as e:
        #If we get an error, we here print the string that failed, to allow debugging.
        print(result)
        pass

### 5.3 Run the analysis

This is the main loop of the code, where we call the LLM for each line in our data, and give it the instructions.

In [333]:
#First, we need to prepare the data and store it in a file for persistency
filename = 'data.pkl'

#These are the columns where we will store the analyzed data
df['answers'] = [[] for _ in range(len(df))]
df['motivations'] = [[] for _ in range(len(df))]

df.to_pickle(filename)

In [363]:
#Main loop 
df = pd.read_pickle(filename)

#If you want to limit the number of lines to analyze
maximum_lines_to_analyze = 100
i = 0

while(True):

    #Find all unprocessed lines
    # left = df.loc[df['result'].isna()]
    left = df.loc[df['answers'].map(len)==0]
    
    #No lines left? Then we're done
    if len(left)==0 or i>= maximum_lines_to_analyze:
        print("All done!")
        break
        
    #Take a random line
    line = left.sample()
    index = line.index.values[0]
    
    print(f"There are {len(left)} left to process. Processing: {index}")
    
    #Wait for a bit, to not overload the API
    time.sleep(WAIT_TIME)
    
    #Analyze the specific line, chunk by chunk
    for chunk in line['text_chunks'].values[0]:
        result = analyze_message(chunk, instruction, model = MODEL)

        #Parse the results, and put into dataframe
        answer,motivation = parse_result(result)

        df.loc[index,'answers'].append(answer)
        df.loc[index,'motivations'].append(motivation)
    
    i+=1
    
    #Save the result to persistent file
    df.to_pickle(filename)

### Post-analysis calculations

Following the LLM analysis, we may need to do some minor calculations or modifications of the results. 

For instance, we need to combine the values returned for the different chunks to a final complete values for the full text. This can be done in several ways, but the most straight-forward is to take the average values for each part. If the text only has one chunk, the result will be used without change. We will here leave the motivations as a list.

In [337]:
#Take the mean level of populism in the text
df['answer'] = [np.mean(answers) if len(answers)>0 else None for answers in df['answers']]

### Example result

We can now look at some examples of the result from the analysis and the associated motivation. 

For instance, we here look at the rating of Donald Trump's inaguration speech:

In [320]:
trump = df.loc[df.filename == 'US_Trump_Famous_1.txt']
print(f"""Rating: {trump.answer.values[0]}. Motivation: '{trump.motivations.values[0][0]}'""")

Rating: 	 1.5. 
Motivation: 	 'The text shows many elements of a populist text. It refers to "the people" and "ordinary people" as a homogeneous community and promotes the idea of politics as the popular will of "the people." It also criticizes the elite and describes them as having reaped the rewards of government while the people have borne the cost. However, it does not use bellicose language or references to a particular enemy.'


At face value, this motivation seems both reasonable and plausible. We will now turn to carry out a more in-depth validation.

# 6. Validation

Finally, we need to validate our results. Careful validation is essential to make sure that the models are measuring what we intend -- and that they do so without problematic biases. To validate our models, we can compare the outputs with established benchmarks, ground truth data, or expert evaluations to validate the effectiveness in achieving the desired analysis outcomes. Validation can furthermore help us fine-tune the model prompt to improve the results.

A simple way of validating can be to output a random sample to an Excel file, and have human coders manually classifying the data to compare the results. To do so, the code below can be used:

### 6.1 Acquire validation data

In [70]:
# # Code to extract the data as excel for manual checking. This is included for illustration, however, we won't use this here.
# sample_size = 100
# sample = df.sample(sample_size).reset_index()
# sample['manual_classification'] = None
# sample[['index','text','manual_classification']].to_excel('manual_validation.xlsx')

# # Now open the resulting file in Excel. Carry out manual classification and put result in the final column

# manual_result = pd.read_excel('manual_validation_finished.xlsx')

In the case of the populism example, however, the Global Populism Database already offers a large sample of manually classified datapoints that we can use for validation. 

We first need to make sure the data is in the right format for running simpledorff. Each line should be one coder response.

In [343]:
#Load and clean the validation data. 
val = pd.read_csv('./global-populism-dataset/gpd_v2_20220427.csv')
val = val[val['merging_variable'].notna()] 
val = val[val['rubricgrade'].notna()] #The database contains some NaN values for the index; we remove these lines
val = val[['merging_variable','codernum','rubricgrade','averagerubric']]

In [344]:
#We include only the lines that we've coded with the LLM
included = set(df.loc[~df['answer'].isna()].filename.values)
val = val.loc[(val['merging_variable'].isin(included)) &  (val['codernum']<=2) ]

#We compare our result with that of the average coder result
val = val.drop_duplicates(subset=['merging_variable'], keep='first')[['merging_variable','averagerubric']].rename(columns={'averagerubric':'answer'})
val['codernum'] = 'human'

#Fit our coded data into the same format to allow processing
df2 = df[['filename','answer','motivations']].dropna(subset=['answer']).rename(columns={'filename':'merging_variable'})
df2['codernum'] = 'llm'

#Combine the two datasets
validation_data = pd.concat([val,df2])

### 6.2 Measure Krippendorf's Alpha

To compare our data against the validation data, we can use Krippendorf's Alpha (see how-to guide for details.) We here use the simpledorff library to do so.

In [ ]:
!pip install simpledorff

In [71]:
import simpledorff

In [350]:
#Calculate inter-coder reliability
#Note that this uses the interval metric. If your variable is categorical, you need to remove the metric_fn parameter.
KA = simpledorff.calculate_krippendorffs_alpha_for_df(test,metric_fn=simpledorff.metrics.interval_metric,experiment_col='merging_variable', annotator_col='codernum', class_col='answer')

print(f"The resulting Krippendorf's Alpha is is {KA}.")

The resulting Krippendorf's Alpha is is 0.6348188209387895.


This is a relatively high value for a first iteration of prompt development for a challenging concept.


### 6.3 Carry out iterative process of concept and prompt development 

Having measured the disagreements between coders and LLM, we can now seek to try to understand the sources of the disagreement. This can be best thought of as a process of mutual learning through which we develop and operationalize a rigorous social scientific concept in the form of a prompt.

We can here work with the coders, and comparing their notes to the motivations given by the LLM, focusing on the examples where the LLM and the human coders are (most) in disagreement. We may find that the prompt can be improved - or that our human coders were mistaken or biased. 

In our case, we do not have access to the coders, and we will simply show the process through which this form of work can be done.

In [ ]:
# We create a dataframe that lists the level of disagreement between coders and LLM
wrong = df2.merge(val, on='merging_variable')
wrong['diff'] = abs(wrong['answer_x']-wrong['answer_y'])

In [ ]:
#We can save as CSV file to analyze results in Excel, or examine the results here.
# display(wrong.sort_values(['diff']))
wrong.sort_values(['diff']).to_csv('disagreements.csv') 

In [360]:
# One of the cases where the LLM and human coders disagree the most is a speech by Berlusconi. The LLM does not think it is populist, but the human coders do.

#The motivation given by the LLM is:
wrong.loc[wrong['merging_variable']=='Italy_Berlusconi_Ribbon_2.txt'].motivations.values[0][0]


' The text does not contain populist elements. It does not focus on "the people" as a homogeneous group, nor does it depict "the elite" as a corrupt entity. Instead, it focuses on historical events, the importance of freedom, and the unity of the nation.'

Here follows the text, translated to English. 

Do you agree with the LLM or the huamn coders? If the latter, how do you think the prompt should be modified to improve the results? 

In [362]:
print("""Dear friends,\n\nIt is not easy to find the words to describe my, our state of mind at this moment. We are gathered here in Onna to celebrate the Liberation Day, a celebration that is both an honor and a commitment.\n\nAn honor: to commemorate a terrible massacre that took place right here in June 1944 when the Nazis, in retaliation, killed 17 citizens of Onna and then blew up the house where the bodies of those innocent victims were found.\n\nA commitment: what should inspire us is not to forget what happened here and to remember the horrors of totalitarianism and the suppression of 'freedom'.\n\nRight here, in Abruzzo, the legendary Maiella Brigade was born and operated, decorated with the Gold Medal for Military Valor. In December '43, 15 young people founded what would become the Maiella Brigade, which grew to 1,500 strong.\n\nIt is no coincidence that on this special day, the soldiers of the Honor Guard standing before us belong to the 33rd Artillery Regiment, the Abruzzesi unit that in 1943 on Cephalonia had the courage to resist the Nazis and sacrifice themselves – fighting – for the honor of our country.\n\nTo those patriots who fought for the redemption and rebirth of Italy, our admiration, gratitude, and recognition must always go.\n\nMost Italians today have not experienced what it means to be deprived of freedom. Only the elderly have a direct memory of totalitarianism, foreign occupation, and the war for the liberation of our homeland.\n\nFor many of us, it is a memory tied to our families, our parents, our grandparents, many of whom were protagonists or victims of those dramatic days. For me, it is the memory of years of separation from my father, forced to emigrate to avoid arrest, the memory of my mother's sacrifices, who alone had to support a large family during those difficult years. It is the memory of her courage, of her, like many others, traveling by train every day from a small town in the province of Como to work in Milan, and on one of those trains, risking her life but managing to save a Jewish woman from the clutches of a Nazi soldier destined for the extermination camps.\n\nThese are the memories, the examples with which we grew up – the memories of a generation of Italians who did not hesitate to choose freedom, even at the risk of their own safety and lives.\n\nOur country owes an inexhaustible debt to those many young people who sacrificed their lives during their most beautiful years to redeem the honor of the nation, out of fidelity to an oath, but above all for that great, splendid, and essential value which is freedom.\n\nWe owe the same debt of gratitude to all those other boys, Americans, English, French, Polish, from the many allied countries, who shed their blood in the Italian campaign. Without them, the sacrifice of our partisans would have risked being in vain.\n\nAnd with respect, we must remember today all the fallen, even those who fought on the wrong side, sincerely sacrificing their lives for their ideals and a lost cause.\n\nThis does not mean, of course, neutrality or indifference. We are – all free Italians are – on the side of those who fought for our freedom, for our dignity, and for the honor of our homeland.\n\nIn recent years, the history of the Resistance has been deepened and discussed. It is a good thing that it happened. The Resistance, along with the Risorgimento, is one of the founding values of our nation, a return to the tradition of freedom. And freedom is a right that comes before laws and the state because it is a natural right that belongs to us as human beings.\n\nHowever, a free nation does not need myths. As with the Risorgimento, we must also remember the dark pages of the civil war, even those in which those who fought on the right side made mistakes and took on blame.\n\nIt is an exercise in truth, in honesty, an exercise that makes the history of those who fought on the right side with selflessness and courage even more glorious.\n\nIt is the history of the many who fought in the Southern army, who, from Cephalonia onwards, redeemed the honor of the uniform with their blood.\n\nIt is the history of martyrs like Salvo D’Acquisto, who did not hesitate to sacrifice his life in exchange for other innocent lives.\n\nIt is the history of our soldiers interned in Germany who chose concentration camps rather than collaborating with the Nazis.\n\nIt is the history of the many who hid their fellow Jewish citizens, saving them from deportation.\n\nAbove all, it is the history of the many, countless unknown heroes who, with small or great acts of daily courage, contributed to the cause of freedom.\n\nEven the Church, I want to remember, played its part with true courage, to prevent odious concepts like race or religious differences from becoming reasons for persecution and death.\n\nSimilarly, we must remember the young Jews of the Jewish Brigade, who came from ghettos all over Europe, took up arms, and fought for freedom.\n\nAt that moment, many Italians of different faiths, cultures, and backgrounds came together to pursue the same great dream – the dream of freedom.\n\nAmong them were very different individuals and groups. Some thought only of freedom, some dreamed of establishing a different social and political order, some considered themselves bound by an oath of loyalty to the monarchy.\n\nBut they all managed to set aside their differences, even the most profound ones, to fight together. The communists and the Catholics, the socialists and the liberals, the actionists and the monarchists, faced with a common tragedy, each wrote a great page of our history. A page on which our Constitution is based, a page on which our freedom is based.\n\nIn the drafting of the Constitution, the wisdom of the political leaders of that time – De Gasperi and Togliatti, Ruini and Terracini, Nenni, Pacciardi, and Parri – managed to channel deep initial divisions towards a single objective.\n\nAlthough clearly the result of compromises, the republican Constitution achieved two noble and fundamental objectives: guaranteeing freedom and creating the conditions for democratic development in the country. It was not a small feat; in fact, it was the best compromise possible at the time.\n\nHowever, the goal of creating a "common" moral conscience for the nation was missed, perhaps premature for those times, so much so that the predominant value for everyone was anti-fascism, but not necessarily anti-totalitarianism. It was a product of history, a compromise useful to avoid the Cold War that vertically divided Italy from degenerating into a civil war with unpredictable outcomes. But the assumption of responsibility and the sense of the State that animated all the political leaders of that time remain a great lesson that would be unforgivable to forget.\n\nToday, 64 years after April 25, 1945, and twenty years after the fall of the Berlin Wall, our task, the task of all, is to finally build a unified national sentiment.\n\nWe must do it together, together, regardless of political affiliation, together, for a new beginning of our republican democracy, where all political parties recognize the greatest value, freedom, and debate in its name for the good and the interest of all.\n\nThe anniversary of the regained freedom is, therefore, an opportunity to reflect on the past, but also to reflect on the present and the future of Italy. If we can do it together from today onwards, we will have rendered a great service not to one\n\nWe have always rejected the idea that our adversary was our enemy. Our religion of freedom demanded it from us and still does. With the same spirit, I am convinced that the time has come for the Liberation Day to become the Day of Freedom, and for this commemoration to shed the character of opposition that revolutionary culture gave it, a character that still 'divides' rather than 'unites'.\n\nI say this with great serenity, without any intention of creating controversy. April 25 was the origin of a new season of democracy, and in democracy, the people's vote deserves absolute respect from everyone.\n\nAfter April 25, the people peacefully voted for the Republic, and the monarchy accepted the people's judgment.\n\nShortly after, on April 18, 1948, the people's choice was once again decisive for our country: with De Gasperi's victory, the Italian people recognized themselves in the Christian and liberal tradition of their history. The 1950s, always with the support of the popular vote, shaped Italy into a democratic, economic, and social reality. Italy became part of Europe and the West, played a role in promoting Atlantic unity and European unity, transforming from a rejected nation to a respected one.\n\nToday, our young people face other challenges: to defend the freedom conquered by their fathers and expand it even further, aware that without freedom, there can be no peace, justice, or well-being.\n\nSome of these challenges are global and see us engaged alongside free nations: the fight against terrorism, the fight against fanatic and repressive fundamentalism, the fight against racism because freedom, dignity, and peace are rights of every human being, 'everywhere' in the world.\n\nThat's why I want to remember the Italian soldiers engaged in peace missions abroad, especially those who have fallen in carrying out this noble mission. There is an ideal continuity between them and all the heroes, Italian and allied, who sacrificed their lives over 60 years ago to give us back freedom, security, and peace.\n\nToday, the teachings of our fathers take on a special value: this April 25 comes just after the great tragedy that struck this land of Abruzzo. Once again, facing the emergency and tragedy, Italians have shown their ability to unite, to overcome differences, demonstrating that they are a great and cohesive people, full of generosity, solidarity, and courage.\n\nLooking at the many Italians who have been engaged here in rescue and reconstruction efforts, I feel proud, once again, even more so, to be Italian and to lead this wonderful country.\n\nToday, Onna is the symbol of our Italy. The earthquake that destroyed it reminds us of the days when invaders destroyed it. Rebuilding it will mean repeating the gesture of its rebirth after Nazi violence.\n\nAnd it is precisely concerning the heroes of then and today that we all have a great responsibility: to set aside any controversy, to look at the interest of the nation, to safeguard the great heritage of freedom that we inherited from our fathers.\n\nTogether, we all have the responsibility and duty to build a future of prosperity, security, peace, and freedom for all.\n\nLong live Italy! Long live the Republic!\n\nLong live April 25, the celebration of all Italians who love freedom and want to remain free!\n\nLong live April 25, the celebration of regained freedom!""")


Dear friends,

It is not easy to find the words to describe my, our state of mind at this moment. We are gathered here in Onna to celebrate the Liberation Day, a celebration that is both an honor and a commitment.

An honor: to commemorate a terrible massacre that took place right here in June 1944 when the Nazis, in retaliation, killed 17 citizens of Onna and then blew up the house where the bodies of those innocent victims were found.

A commitment: what should inspire us is not to forget what happened here and to remember the horrors of totalitarianism and the suppression of 'freedom'.

Right here, in Abruzzo, the legendary Maiella Brigade was born and operated, decorated with the Gold Medal for Military Valor. In December '43, 15 young people founded what would become the Maiella Brigade, which grew to 1,500 strong.

It is no coincidence that on this special day, the soldiers of the Honor Guard standing before us belong to the 33rd Artillery Regiment, the Abruzzesi unit that in 194